In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
import litellm
import gradio as gr

In [ ]:
# Initialization
# This one line reads your .env file and loads your GEMINI_API_KEY , scince i have my key in .env file.
load_dotenv()

print("✅ API Key loaded securely from .env file.")
    
MODEL = 'gemini/gemini-1.5-flash'


#if you dont have key in .env file you can still use code just download this ipynb file and write prompt to any AI frontier llm model ""Hello, I'm providing my Jupyter notebook code that uses litellm. Please convert it into two separate versions, one for the native OpenAI SDK and another for the native Claude SDK, showing the key code changes. .


In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = litellm.completion(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [ ]:
# when you will launch gradio and ask question about real time price ticket it will dont answer you 
# gr.ChatInterface(fn=chat, type="messages").launch()
# so now we will write tool 

# Tools

In [ ]:
# Let's start by making a useful function
# Mock database for ticket prices 
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")


# Mock database for flight availability and bookings
flight_availability = {
    "london": [
        {"flight_number": "FA101", "departure": "10:00 AM", "seats_available": 5},
        {"flight_number": "FA102", "departure": "09:00 PM", "seats_available": 2},
    ],
    "paris": [
        {"flight_number": "FA201", "departure": "11:30 AM", "seats_available": 10},
    ],
    "tokyo": [
        {"flight_number": "FA301", "departure": "08:00 PM", "seats_available": 0},
    ],
    "berlin": [
        {"flight_number": "FA401", "departure": "01:00 PM", "seats_available": 8},
    ]
}

def get_flight_availability(destination_city):
    """Checks for available flights to a given destination."""
    print(f"Tool get_flight_availability called for {destination_city}")
    city = destination_city.lower()
    return flight_availability.get(city, "No flights found for this destination.")

def book_flight(flight_number, passenger_name):
    """Books a flight for a passenger and returns a confirmation."""
    print(f"Tool book_flight called for flight {flight_number} for {passenger_name}")
    for city, flights in flight_availability.items():
        for flight in flights:
            if flight["flight_number"] == flight_number:
                if flight["seats_available"] > 0:
                    flight["seats_available"] -= 1  # Decrement seat count
                    return f"Booking confirmed for {passenger_name} on flight {flight_number}."
                else:
                    return f"Sorry, flight {flight_number} is fully booked."
    return f"Flight with number {flight_number} not found."



In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
availability_function = {
    "name": "get_flight_availability",
    "description": "Get the available flights for a destination city. Call this when a customer asks about flight times or availability.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to check for flights.",
            },
        },
        "required": ["destination_city"],
    }
}

booking_function = {
    "name": "book_flight",
    "description": "Book a flight for a passenger. Call this when a customer explicitly asks to book a specific flight.",
    "parameters": {
        "type": "object",
        "properties": {
            "flight_number": {
                "type": "string",
                "description": "The flight number for the booking, e.g., 'FA101'.",
            },
            "passenger_name": {
                "type": "string",
                "description": "The full name of the passenger for the booking.",
            },
        },
        "required": ["flight_number", "passenger_name"],
    }
}



In [ ]:
# And this is included in a list of tools:

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": availability_function},
    {"type": "function", "function": booking_function}
]

In [ ]:


def handle_tool_call(message):
    """
    Handles the tool call requested by the LLM.
    It checks which function to call and executes it with the provided arguments.
    """
    tool_call = message.tool_calls[0]
    tool_function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    print(f"LLM wants to call the tool: {tool_function_name}")

    # Initialize tool_response to None
    tool_response = None

    # Route to the correct function based on the tool name
    if tool_function_name == "get_ticket_price":
        destination = arguments.get('destination_city')
        price = get_ticket_price(destination)
        tool_response = {"destination_city": destination, "price": price}

    elif tool_function_name == "get_flight_availability":
        destination = arguments.get('destination_city')
        availability = get_flight_availability(destination)
        tool_response = {"destination_city": destination, "available_flights": availability}

    elif tool_function_name == "book_flight":
        flight_number = arguments.get('flight_number')
        passenger_name = arguments.get('passenger_name')
        confirmation = book_flight(flight_number, passenger_name)
        tool_response = {"flight_number": flight_number, "status": confirmation}

    if tool_response is None:
        # Handle cases where the tool name is not recognized
        print(f"Error: Tool '{tool_function_name}' not found.")
        tool_response = {"error": f"Tool '{tool_function_name}' not found."}

    # Format the final response for the LLM
    response = {
        "role": "tool",
        "content": json.dumps(tool_response),
        "tool_call_id": tool_call.id
    }
    return response

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = litellm.completion(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":  #if model have not answer then use tools(code) written on my machine
        message = response.choices[0].message #collect response form model
        print(message) #to see what was message by model .....optional...
        #response, city = handle_tool_call(message) #unpack response from model , we used a function handel_tool_call to make our code less messy.
        response = handle_tool_call(message)
        messages.append(message) #add two more rows one is message collected from mdodel 
        messages.append(response) # second is response when we use tools
        print(response) # To see response from user after running tool .....optional.....
        response = litellm.completion(model=MODEL, messages=messages) #give final output result

# workflow is like we know user say this asistant replie , so on ... when assitant say i dont have real time price of ticket we use tool and \
# give as user say this then assistant take our response and replied 
    
    return response.choices[0].message.content